In [5]:
!python -c "import torch; print(torch.__version__)"
!python -c "import torch; print(torch.version.cuda)"
!pip install torch-geometric-temporal

2.0.1+cu118
11.8
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 30.1 MB/s eta 0:00:00
  Using cached torch_sparse-0.6.17.tar.gz (209 kB)
  Preparing metadata (setup.py) ... done
  Using cached torch_scatter-2.1.1.tar.gz (107 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 37.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric-temporal: filename=torch_geometric_temporal-0.54.0-py3-none-any.whl size=86722 sha256=97af22abd717c2e9eefd4b19cc952aabd4cf0f8f74fed06034a1be134024c62b
  Stored in directory: /root/.cache/pip/wheels/9e/9b/b6/e15256e053f0cb49b1084a67a709db909d418386a231f0722c
  Created wheel for torch_geometric: filename=torch_geometric-2.3.1-py3-none-any.whl si

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric_temporal.nn.recurrent import A3TGCN2
from torch_geometric_temporal.signal import temporal_signal_split

# GPU support
DEVICE = torch.device('cuda') # cuda
shuffle=True
batch_size = 32

In [9]:
try:
    from tqdm import tqdm
except ImportError:
    def tqdm(iterable):
        return iterable

import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import TGCN

from torch_geometric_temporal.dataset import ChickenpoxDatasetLoader
from torch_geometric_temporal.signal import temporal_signal_split

loader = ChickenpoxDatasetLoader()

dataset = loader.get_dataset()

train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.2)

class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features):
        super(RecurrentGCN, self).__init__()
        self.recurrent = TGCN(node_features, 32)
        self.linear = torch.nn.Linear(32, 1)

    def forward(self, x, edge_index, edge_weight, prev_hidden_state):
        h = self.recurrent(x, edge_index, edge_weight, prev_hidden_state)
        y = F.relu(h)
        y = self.linear(y)
        return y, h

model = RecurrentGCN(node_features = 4)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.train()

for epoch in tqdm(range(50)):
    cost = 0
    hidden_state = None
    for time, snapshot in enumerate(train_dataset):
        y_hat, hidden_state = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr,hidden_state)
        cost = cost + torch.mean((y_hat-snapshot.y)**2)
    cost = cost / (time+1)
    cost.backward()
    optimizer.step()
    optimizer.zero_grad()

model.eval()
cost = 0
hidden_state = None
for time, snapshot in enumerate(test_dataset):
    y_hat, hidden_state = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr, hidden_state)
    cost = cost + torch.mean((y_hat-snapshot.y)**2)
cost = cost / (time+1)
cost = cost.item()
print("MSE: {:.4f}".format(cost))

100%|██████████| 50/50 [00:20<00:00,  2.50it/s]


MSE: 1.0027


In [67]:
# prompt: mount google drive

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [74]:
import os
import json
import numpy as np
from torch_geometric.data import Data

class ASLDatasetLoader:
    def __init__(self, directory_path):
        self.directory_path = directory_path
        self.sign_to_label = self._create_sign_to_label_map()

    def _create_sign_to_label_map(self):
        signs = [os.path.splitext(filename)[0] for filename in os.listdir(self.directory_path)]
        return {sign: i for i, sign in enumerate(signs)}

    def _read_file_data(self, file_path):
        with open(file_path, 'r') as f:
            return json.load(f)

    def _augment_data(self, frame_data, rotation_range=10, translation_range=0.05, scaling_range=0.1):
        """
        Augment the frame data with random rotation, translation, and scaling.

        :param frame_data: Dictionary containing frame landmarks and deltas.
        :param rotation_range: Maximum rotation angle in degrees.
        :param translation_range: Maximum translation as a fraction of landmark range.
        :param scaling_range: Maximum scaling factor.
        :return: Augmented frame data.
        """
        landmarks = np.array(frame_data["landmarks"])
        centroid = np.mean(landmarks, axis=0)

        # Random rotation
        theta = np.radians(np.random.uniform(-rotation_range, rotation_range))
        rotation_matrix = np.array([
            [np.cos(theta), -np.sin(theta)],
            [np.sin(theta), np.cos(theta)]
        ])
        landmarks = np.dot(landmarks - centroid, rotation_matrix) + centroid

        # Random translation
        max_translation = translation_range * (landmarks.max(axis=0) - landmarks.min(axis=0))
        translations = np.random.uniform(-max_translation, max_translation)
        landmarks += translations

        # Random scaling
        scale = np.random.uniform(1 - scaling_range, 1 + scaling_range)
        landmarks = centroid + scale * (landmarks - centroid)

        frame_data["landmarks"] = landmarks.tolist()
        return frame_data


    def _create_graph_from_frame(self, sign_name, frame_data):
        landmarks = frame_data["landmarks"]
        deltas = frame_data["deltas"]

        # Create edges assuming landmarks are connected in sequence.
        # For a more complex hand structure, this needs to be adapted.
        edges = [[i, i+1] for i in range(len(landmarks) - 1)]

        edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
        x = torch.tensor(np.hstack((landmarks, deltas)), dtype=torch.float)
        y = torch.tensor([self.sign_to_label[sign_name]], dtype=torch.long)

        return Data(x=x, edge_index=edge_index, y=y)

    def get_dataset(self, augment=False):
        dataset = []

        for filename in os.listdir(self.directory_path):
            sign_name = os.path.splitext(filename)[0]
            file_path = os.path.join(self.directory_path, filename)
            sign_data = self._read_file_data(file_path)

            for frame_data in sign_data["frames"]:
                if augment:
                  frame_data = self._augment_data(frame_data)
                graph_data = self._create_graph_from_frame(sign_name, frame_data)

                dataset.append(graph_data)

        return dataset

    def number_of_classes(self):
        return len(self.sign_to_label)

In [88]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool, BatchNorm

class GraphClassifier(torch.nn.Module):
    def __init__(self, num_node_features, num_classes):
        super(GraphClassifier, self).__init__()
        self.conv1 = GCNConv(num_node_features, 128)
        self.bn1 = BatchNorm(128)  # Batch Normalization after the first layer
        self.conv2 = GCNConv(128, 64)
        self.bn2 = BatchNorm(64)  # Batch Normalization after the second layer
        self.fc = torch.nn.Linear(64, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # First GCN layer with Batch Normalization
        x = self.conv1(x, edge_index)
        x = self.bn1(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)

        # Second GCN layer with Batch Normalization
        x = self.conv2(x, edge_index)
        x = self.bn2(x)
        x = F.relu(x)

        # Global pooling across nodes
        x = global_mean_pool(x, data.batch)

        # Final classification layer
        x = self.fc(x)

        return F.log_softmax(x, dim=1)


In [89]:
from collections import Counter
import random


def train():
    directory_path = "/content/drive/MyDrive/Colab Notebooks/DGMD E-14 Project/Datasets/ASL"
    loader = ASLDatasetLoader(directory_path)

    # Create the entire dataset without augmentation
    data_list = loader.get_dataset()

    # Shuffle and split
    random.shuffle(data_list)
    train_size = int(0.8 * len(data_list))
    train_dataset = data_list[:train_size]
    test_dataset = data_list[train_size:]

    # Now augment only the training dataset
    augmented_train_dataset = loader.get_dataset(augment=True)
    train_dataset = augmented_train_dataset[:train_size]  # Only take the initial portion corresponding to training

    num_classes = loader.number_of_classes()

    # Debugging code starts here
    train_labels = [data.y.item() for data in train_dataset]
    test_labels = [data.y.item() for data in test_dataset]

    print("Training label distribution:", Counter(train_labels))
    print("Test label distribution:", Counter(test_labels))
    # Debugging code ends here

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # Adjust the number of node features based on your dataset
    model = GraphClassifier(num_node_features=4, num_classes=num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)

    model.train()
    for epoch in range(100):
        total_loss = 0
        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            out = model(batch)
            loss = F.nll_loss(out, batch.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch}, Loss: {total_loss / len(train_loader)}")

    model.eval()
    correct = 0
    all_preds = []
    all_labels = []

    for batch in test_loader:
        batch = batch.to(device)
        with torch.no_grad():
            pred = model(batch).max(dim=1)[1]
            all_preds.extend(pred.cpu().numpy())
            all_labels.extend(batch.y.cpu().numpy())
            correct += pred.eq(batch.y).sum().item()

    print(f"Accuracy: {correct / len(test_dataset)}")

    # Print a few predictions and true labels for inspection:
    print("Sample predictions:", all_preds[:20])
    print("Sample true labels:", all_labels[:20])

In [90]:
train()

Training label distribution: Counter({0: 502, 3: 336, 5: 331, 6: 298, 2: 279, 4: 269, 1: 249})
Test label distribution: Counter({0: 115, 5: 70, 6: 63, 7: 60, 2: 58, 3: 56, 8: 55, 1: 48, 4: 42})
Epoch 0, Loss: 1.8325641306353286
Epoch 1, Loss: 1.511253875745854
Epoch 2, Loss: 1.3824609249410496
Epoch 3, Loss: 1.2959549309502185
Epoch 4, Loss: 1.2174891283814335
Epoch 5, Loss: 1.1384455771513389
Epoch 6, Loss: 1.114060253324643
Epoch 7, Loss: 1.0601134963438545
Epoch 8, Loss: 1.034084915275305
Epoch 9, Loss: 1.0141520323887678
Epoch 10, Loss: 0.9802168834377343
Epoch 11, Loss: 0.9556333884386949
Epoch 12, Loss: 0.9539522654573682
Epoch 13, Loss: 0.9377628514464472
Epoch 14, Loss: 0.9176177709874972
Epoch 15, Loss: 0.9044702833806965
Epoch 16, Loss: 0.9157127418988188
Epoch 17, Loss: 0.8644599352084416
Epoch 18, Loss: 0.8486766622100078
Epoch 19, Loss: 0.8565347261831794
Epoch 20, Loss: 0.8524264919925744
Epoch 21, Loss: 0.8538088219266542
Epoch 22, Loss: 0.8363908285826025
Epoch 23, Loss